In [25]:
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [26]:
import pandas as pd
import numpy as np
import re
from functools import partial
import pickle

In [27]:
students_raw = pd.read_csv("../data/students_dropout_raw.csv", sep=";")

In [28]:
students_raw.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [29]:
students_raw.columns

Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Displaced',
       'Educational special needs', 'Debtor', 'Tuition fees up to date',
       'Gender', 'Scholarship holder', 'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd sem (grade)',
       'Curricular units 2nd sem (without evaluations)

In [30]:
def names_cleaning_1(df):
    df = df.copy()
    # Rename the columns
    df = df.rename(columns=lambda x: x.replace(" ", "_").lower())
    # Rename the nationality column, spelled 'nacionality' in the original data
    df = df.rename(columns={"nacionality": "nationality"})
    df["dropout"] = np.where(df["target"] == "Dropout", 1, 0)
    df = df.drop(columns="target")
    return df

In [31]:
students = students_raw.pipe(names_cleaning_1)

Let's match the categorical columns with the original categories, loading the dictionary scraped from the site

In [32]:
with open("../data/categorical_maps.pkl", "rb") as f:
    data_map = pickle.load(f)

In [33]:
cat_columns = [k for k, v in data_map.items() if not isinstance(v, str)]
num_columns_dict = {k: v for k, v in data_map.items() if isinstance(v, str)}

In [34]:
cat_columns

['marital_status',
 'nationality',
 'displaced',
 'gender',
 'international',
 "father's_qualification",
 "mother's_qualification",
 "father's_occupation",
 "mother's_occupation",
 'educational_special_needs',
 'debtor',
 'tuition_fees_up_to_date',
 'scholarship_holder',
 'application_mode',
 'course',
 'daytime/evening_attendance',
 'previous_qualification']

In [35]:
num_columns_dict

{'age_at_enrollment': 'Age of students at enrollment',
 'unemployment_rate': 'Unemployment rate (%)',
 'inflation_rate': 'Inflation rate (%)',
 'gdp': 'Gross domestic product',
 'application_order': 'Application order (between 0 - first choice; and 9 last choice)',
 'curricular_units_1st_sem_(credited)': 'Number of curricular units credited in the 1st semester',
 'curricular_units_1st_sem_(enrolled)': 'Number of curricular units enrolled in the 1st semester',
 'curricular_units_1st_sem_(evaluations)': 'Number of evaluations to curricular units in the 1st semester',
 'curricular_units_1st_sem_(approved)': 'Number of curricular units approved in the 1st semester',
 'curricular_units_1st_sem_(grade)': 'Grade average in the 1st semester (between 0 and 20)',
 'curricular_units_1st_sem_(without_evaluations)': 'Number of curricular units without evaluations in the 1st semester',
 'curricular_units_2nd_sem_(credited)': 'Number of curricular units credited in the 2nd semester',
 'curricular_uni

In [36]:
set(cat_columns).issubset(set(students.columns))

True

This mean that all the columns in our categorical columns dictionary are present in the data, that's a good sign. Now I'm going to replace the numerical values for the original categories of the data

In [37]:
def map_categories(df, cat_columns, mapping_dict):
    df = df.copy()
    for col in cat_columns:
        df[col] = df[col].map(mapping_dict.get(col))
    return df

In [38]:
students_cat = students.pipe(map_categories, cat_columns, data_map)

Now that we have our original categories back, let's perform a further clean of the column names

In [39]:
def names_cleaning_2(df):
    df = df.copy()
    df = df.rename(
        columns={
            "daytime/evening_attendance": "day/evening",
            "educational_special_needs": "special_needs",
            "gdp": "gdp_change",
        }
    )
    df = df.rename(
        columns=lambda x: x.replace("'s", "")
        .replace("curricular_units_", "")
        .replace("1st", "first")
        .replace("2nd", "second")
        .replace("qualification", "education")
        .replace("/", "_")
    )
    df = df.rename(columns=lambda x: re.sub(r"\((.*)\)", r"\1", x))
    return df

In [40]:
students_cat = students_cat.pipe(names_cleaning_2)

In [41]:
# Number of categories in the parents occupation and qualification columns

print(
    f'The father occupation column have {students_cat["father_occupation"].nunique()} distinct categories'
)
print(
    f'The mother occupation column have {students_cat["mother_occupation"].nunique()} distinct categories'
)
print(
    f'The father education column have {students_cat["father_education"].nunique()} distinct categories'
)
print(
    f'The mother education column have {students_cat["mother_education"].nunique()} distinct categories'
)
print(
    f'The previous education column have {students_cat["previous_education"].nunique()} distinct categories'
)

The father occupation column have 46 distinct categories
The mother occupation column have 32 distinct categories
The father education column have 34 distinct categories
The mother education column have 29 distinct categories
The previous education column have 17 distinct categories


These are too many different values either for data visualization or machine learning. I will lump some categories together through dictionaries. Also, I will order to the education column categories based on years of education

In [42]:
education_map = {
    "Higher Education": "Higher Education",
    "Secondary Education|12th": "Secondary Education",
    "Basic education 1st cycle|4th|5th": "Basic education 1st cycle",
    "Basic Education 3rd Cycle|9th|10th|11th": "Basic Education 3rd Cycle",
    "Basic Education 2nd Cycle|6th|7th|8th": "Basic Education 2nd Cycle",
    "Unknown|Can't read": "Unknown",
    "2nd cycle of the general high school": "Basic Education 3rd Cycle",
    "course|Supplementary": "Tertiary course",
}

education_order = [
    "Unknown",
    "Basic education 1st cycle",
    "Basic Education 2nd Cycle",
    "Basic Education 3rd Cycle",
    "Secondary Education",
    "Tertiary course",
    "Higher Education",
]

occupation_map = {
    "Unskilled": "Unskilled",
    "Administrative": "Administrative",
    "Skilled": "Skilled",
    "Technicians": "Technicians",
    "Specialists": "Specialists",
    "Operators": "Operators",
    "Personal Services, Security and Safety Workers and Sellers": "Personal Services, Security and Safety Workers and Sellers",
    "Managers": "Managers",
    "Armed Forces Professions": "Armed Forces",
    "Student": "Student",
}

In [43]:
def map_values(x, map_dict):
    try:
        return next(
            map_dict[k] for k in map_dict if re.search(k, x, flags=re.IGNORECASE)
        )
    except:
        return "Other"


map_education = partial(map_values, map_dict=education_map)
map_occupation = partial(map_values, map_dict=occupation_map)

In [44]:
students_cat["father_occupation"] = students_cat["father_occupation"].map(
    map_occupation
)
students_cat["mother_occupation"] = students_cat["mother_occupation"].map(
    map_occupation
)
students_cat["father_education"] = students_cat["father_education"].map(map_education)
students_cat["mother_education"] = students_cat["mother_education"].map(map_education)
students_cat["previous_education"] = students_cat["previous_education"].map(
    map_education
)

In [45]:
# Check the new number of distinct categories

print(
    f'The father occupation column have {students_cat["father_occupation"].nunique()} distinct categories'
)
print(
    f'The mother occupation column have {students_cat["mother_occupation"].nunique()} distinct categories'
)
print(
    f'The father education column have {students_cat["father_education"].nunique()} distinct categories'
)
print(
    f'The mother education column have {students_cat["mother_education"].nunique()} distinct categories'
)
print(
    f'The previous education column have {students_cat["previous_education"].nunique()} distinct categories'
)

The father occupation column have 11 distinct categories
The mother occupation column have 11 distinct categories
The father education column have 7 distinct categories
The mother education column have 7 distinct categories
The previous education column have 5 distinct categories


In [46]:
students_cat.previous_education.value_counts()

Secondary Education          3728
Tertiary course               255
Higher Education              220
Basic Education 3rd Cycle     214
Basic Education 2nd Cycle       7
Name: previous_education, dtype: int64

In [47]:
# Order the education columns
students_cat["father_education"] = (
    students_cat["father_education"]
    .astype("category")
    .cat.reorder_categories(education_order, ordered=True)
)
students_cat["mother_education"] = (
    students_cat["mother_education"]
    .astype("category")
    .cat.reorder_categories(education_order, ordered=True)
)

In [48]:
students_cat.course.unique()

array(['Animation and Multimedia Design', 'Tourism',
       'Communication Design', 'Journalism and Communication',
       'Social Service (evening attendance)',
       'Management (evening attendance)', 'Nursing', 'Social Service',
       'Advertising and Marketing Management', 'Basic Education',
       'Veterinary Nursing', 'Equiniculture', 'Oral Hygiene',
       'Management', 'Agronomy', 'Biofuel Production Technologies',
       'Informatics Engineering'], dtype=object)

We can see that the evening courses have a `(evening attendace)` tag in their name. I will remove this since there is a column in the data to indicate this condition

In [49]:
students_cat["course"] = students_cat["course"].str.replace(
    " (evening attendance)", "", regex=False
)

In [50]:
students_cat.course.unique()

array(['Animation and Multimedia Design', 'Tourism',
       'Communication Design', 'Journalism and Communication',
       'Social Service', 'Management', 'Nursing',
       'Advertising and Marketing Management', 'Basic Education',
       'Veterinary Nursing', 'Equiniculture', 'Oral Hygiene', 'Agronomy',
       'Biofuel Production Technologies', 'Informatics Engineering'],
      dtype=object)

In [51]:
# Save the processed file to parquet, this keeps the format in the columns (ie: keeps the order created for some categorical columns)
students_cat.to_parquet("../data/students_processed_cats.parquet", index=False)

In [53]:
# Split the data into train and test sets for machine learning models analysis

from sklearn.model_selection import train_test_split

train, test = train_test_split(
    students_cat,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=students_cat["dropout"],
)
train.to_parquet("../data/splits/train.parquet", index=False)
test.to_parquet("../data/splits/test.parquet", index=False)